In [ ]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# import torch
# image_size = 640
# predicted_masks = torch.zeros((2, 3, 512, 512))
# predicted_masks = torch.nn.functional.interpolate(predicted_masks, size=(image_size, image_size), mode="bilinear", align_corners=False)
# predicted_masks.shape

In [ ]:
# import torch
# from transformers import SegformerForSemanticSegmentation
# import json
# from huggingface_hub import hf_hub_download

# # define model
# model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b1",
#                                                          num_labels=1,
#                                                          id2label={1: "solar_pv"},
#                                                          label2id={"solar_pv": 1},
# ).cuda()
# inputs = torch.zeros((1, 3, 512, 512))
# model.eval()
# model(inputs.to("cuda")).logits.shape

In [ ]:
DATADIR="/content/drive/MyDrive/solar_PV/data/"
SRCDIR="/content/drive/MyDrive/solar_PV_prediction/src/"

In [ ]:
# Installing the environment
!pip install monai
!pip install wandb
!pip install segmentation-models-pytorch
!pip install pandas -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
# import os
# import sys
# import segmentation_models_pytorch as smp

# sys.path.append(SRCDIR)

# import data

# model = smp.DeepLabV3Plus(
#     encoder_name='resnet101',
#     encoder_weights='imagenet',
#     classes=1,
#     activation='sigmoid',
# )
# preprocessing_fn = smp.encoders.get_preprocessing_fn('resnet101', 'imagenet')
# model = model.to("cuda")
# image_size, mask_size = 512, 512
# val_csv = os.path.join("/content/drive/MyDrive/solar_PV/src/dataset_csv", "val.csv")
# val_data = data.SegmentationDataset(val_csv,
#                                     image_size=image_size,
#                                     mask_size=mask_size,
#                                     transform=preprocessing_fn, model_name="deeplabv3-resnet101")
# inputs = val_data[0]['pixel_values']
# inputs.max(), inputs.min()
# # import torch
# # inputs = torch.zeros((1, 3, 512, 512))
# model.eval()
# model((inputs.unsqueeze(0)).to("cuda"))

In [ ]:
# inputs.max(), inputs.min()

In [ ]:
import os
import sys
import wandb
import pandas as pd
from datetime import datetime

sys.path.append(SRCDIR)

import numpy as np
import params as args # type: ignore
import losses, metrics, data, models, training # type: ignore
args.batch_size = 4
args.lr = 0.00002
args.training_ratio=1
args.optim = "adam"
args.loss = "Dice+BCE"
args.resume_ckpt ="/content/drive/MyDrive/solar_PV_prediction/outputs_segAug/segformer-b5_25-06-20_lr_2e-05_opt_adam_bsz_4_ratio_1_loss_Dice+BCE_country_China/model_39.pth"
args.model_name = "segformer-b5"# deeplabv3-resnet101", "sam""segformer-b5""unet"
args.dataset_csv = "/content/drive/MyDrive/solar_PV_prediction/src/dataset_csv"
country = "China"
if country != "All":
  train_csv = os.path.join(args.dataset_csv, f"train_{country}.csv")
  print(pd.read_csv(train_csv).shape)
else:
  train_csv = os.path.join(args.dataset_csv, f"train.csv")
val_csv = os.path.join(args.dataset_csv, "val.csv")
test_csv = os.path.join(args.dataset_csv, "test.csv")
args.exp_date = datetime.now().strftime("%y-%m-%d")
args.exp_code = f'{args.model_name}_{args.exp_date}_lr_{args.lr}_opt_{args.optim}_bsz_{args.batch_size}_ratio_{args.training_ratio}_loss_{args.loss}_country_{country}'
print(args)

if args.report_to == "wandb":
  writer = wandb.init(project="Solar-PV-segmentation-finetuning-" + args.exp_code)
else:
  writer = None

(589, 3)
<module 'params' from '/content/drive/MyDrive/solar_PV_prediction/src/params.py'>


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peijinjiang1201 (hansen_msr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
args.__dict__

{'__name__': 'params',
 '__doc__': None,
 '__package__': '',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7cd38d8bea50>,
 '__spec__': ModuleSpec(name='params', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7cd38d8bea50>, origin='/content/drive/MyDrive/solar_PV_prediction/src/params.py'),
 '__file__': '/content/drive/MyDrive/solar_PV_prediction/src/params.py',
 '__cached__': '/content/drive/MyDrive/solar_PV_prediction/src/__pycache__/params.cpython-311.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built-in value, but in\nwhich the built-in of that name is also needed.",
  '__p

In [ ]:
training.train_model(train_csv, val_csv, test_csv, writer, args)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/328M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Batch 140 | Loss: 0.0432 | Learning rate:  0.000001


Start training the model for one epoch...: 100%|██████████| 148/148 [01:03<00:00,  2.31it/s]
954it [02:08,  7.44it/s]


Epoch: 199, train loss: 0.029965614522429736, val_loss: 0.3765310122000846, val_dice: 0.14200405034331492, val_IoU: 0.1262339407141692


25it [00:10,  2.47it/s]

Epoch: 200, test_loss: 0.13355679288506508, test_dice: 0.7233974809224515, test_IoU: 0.645023427125914


Run variance

In [ ]:

import os
import sys
import wandb
import numpy as np
import pandas as pd
from datetime import datetime

sys.path.append(SRCDIR)
import torch
import params as args # type: ignore
import losses, metrics, data, models, training # type: ignore

from tqdm import tqdm
from torch.utils.data import DataLoader


def segmentation_metrics(masks, pred):
    """
    Calculate common segmentation metrics: Dice coefficient, IoU, Pixel Accuracy, Precision, and Recall.

    Parameters:
    masks (numpy.ndarray): Ground truth masks, shape (N, W, H).
    pred (numpy.ndarray): Predicted masks, shape (N, W, H).

    Returns:
    dict: A dictionary with Dice, IoU, Pixel Accuracy, Precision, and Recall for each mask.
    """

    def dice_coefficient(mask, pred):
        """Calculate the Dice coefficient."""
        intersection = np.sum(mask * pred)
        union = np.sum(mask) + np.sum(pred)
        dice = (2 * intersection) / (union + 1e-6)  # Add small epsilon to avoid division by zero
        return dice

    def iou(mask, pred):
        """Calculate the Intersection over Union (IoU)."""
        intersection = np.sum(mask * pred)
        union = np.sum(mask) + np.sum(pred) - intersection
        iou_score = intersection / (union + 1e-6)  # Add small epsilon to avoid division by zero
        return iou_score

    def pixel_accuracy(mask, pred):
        """Calculate the pixel accuracy."""
        correct = np.sum(mask == pred)
        total = mask.size
        accuracy = correct / total
        return accuracy

    def precision(mask, pred):
        """Calculate the precision."""
        TP = np.sum((mask == 1) & (pred == 1))
        FP = np.sum((mask == 0) & (pred == 1))
        precision_score = TP / (TP + FP + 1e-6)  # Add small epsilon to avoid division by zero
        return precision_score

    def recall(mask, pred):
        """Calculate the recall."""
        TP = np.sum((mask == 1) & (pred == 1))
        FN = np.sum((mask == 1) & (pred == 0))
        recall_score = TP / (TP + FN + 1e-6)  # Add small epsilon to avoid division by zero
        return recall_score

    metrics = {'dice': [], 'iou': [], 'pixel_accuracy': [], 'precision': [], 'recall': []}

    for i in range(masks.shape[0]):
        mask = masks[i]
        pred_mask = pred[i]
        metrics['dice'].append(dice_coefficient(mask, pred_mask))
        metrics['iou'].append(iou(mask, pred_mask))
        metrics['pixel_accuracy'].append(pixel_accuracy(mask, pred_mask))
        metrics['precision'].append(precision(mask, pred_mask))
        metrics['recall'].append(recall(mask, pred_mask))

    # Compute the mean of each metric
    metrics = {k: v for k, v in metrics.items()}
    return metrics


def eval_bootstrap(model, val_dl, image_size, args):
    model.eval()
    val_stats = {}
    preds_gather, labels_gather = [], []

    with torch.no_grad():
        for batch_idx, val_batch in tqdm(enumerate(val_dl)):
            if "sam" in args.model_name:
              outputs = model(
                  pixel_values=val_batch["pixel_values"].to(args.device),
                  input_boxes=val_batch["input_boxes"].to(args.device),
                  multimask_output=False,
              )
              predicted_masks = outputs.pred_masks.squeeze()
            elif "segformer" in args.model_name:
              outputs = model(val_batch["pixel_values"].to(args.device))
              predicted_masks = outputs.logits.squeeze()
            else:
              outputs = model(val_batch["pixel_values"].to(args.device))
              predicted_masks = outputs.squeeze()
            ground_truth_masks = val_batch["ground_truth_mask"].float().to(args.device)
            if len(predicted_masks.shape) == 2:
              predicted_masks = predicted_masks.unsqueeze(0)
            preds_gather.append(predicted_masks.detach().cpu())
            labels_gather.append(ground_truth_masks.detach().cpu())


    preds_gather = (torch.cat(preds_gather, dim=0) > 0.0).cpu().numpy()
    labels_gather = torch.cat(labels_gather, dim=0).bool().cpu().numpy()

    n_bootstrap = 1000
    seg_metrics = segmentation_metrics(preds_gather, labels_gather)
    for _ in tqdm(range(n_bootstrap), desc="Start bootstrapping..."):
      sample_indices = np.random.choice(np.arange(len(preds_gather)), size=len(preds_gather), replace=True)

      for met in seg_metrics:
          if met not in val_stats:
            val_stats[met] = []
          vs = np.asarray(seg_metrics[met])[sample_indices]
          val_stats[met].append(np.mean(vs))
    return val_stats

def get_variance(test_path, args, model_path=""):

    model, processor, image_size, mask_size = models.load_model(args.model_name, args.device)
    test_data = data.SegmentationDataset(test_path, image_size=image_size, mask_size=mask_size, transform=processor, model_name=args.model_name)
    test_dl = DataLoader(test_data, batch_size=args.batch_size, num_workers=args.workers, shuffle=False)

    best_val_loss, best_val_model = None, None
    model.load_state_dict(torch.load(model_path))
    print(f"Resume from previous checkpoint: {model_path}...")

    test_stats = eval_bootstrap(model, test_dl, image_size, args)
    for k, vs in test_stats.items():
      print(f"{k}, mean: {np.mean(vs)}, variance: {np.std(vs)}")


args.model_name = "unet"# segformer-b5", "sam""segformer-b5""unet"
args.dataset_csv = "/content/drive/MyDrive/solar_PV_prediction/Japan/zoom19_cv/segmentation"
test_csv = os.path.join(args.dataset_csv, "test.csv")
model_path = "/content/drive/MyDrive/solar_PV_prediction/outputs_segAug/unet_25-06-25_lr_5e-05_opt_adamw_bsz_16_ratio_1_loss_Dice+BCE_country_France/model_39.pth" # PUT YOUR MODEL PATH HERE
get_variance(test_csv, args, model_path)

Resume from previous checkpoint: /content/drive/MyDrive/solar_PV_prediction/outputs_segAug/unet_25-06-25_lr_5e-05_opt_adamw_bsz_16_ratio_1_loss_Dice+BCE_country_France/model_39.pth...


50it [00:01, 25.10it/s]
Start bootstrapping...: 100%|██████████| 1000/1000 [00:00<00:00, 13034.05it/s]

dice, mean: 0.697186278615669, variance: 0.027923907061107707
iou, mean: 0.5956116761152563, variance: 0.028224953762359475
pixel_accuracy, mean: 0.9835951514053345, variance: 0.0025510199614483336
precision, mean: 0.822040537060724, variance: 0.02675773324714518
recall, mean: 0.644416176274117, variance: 0.029893139828934906


Run test

In [ ]:
# import os
# import sys
# import wandb
# import pandas as pd
# from datetime import datetime

# sys.path.append(SRCDIR)

# import numpy as np
# import params as args # type: ignore
# import losses, metrics, data, models, training # type: ignore

# args.exp_date = datetime.now().strftime("%y-%m-%d")
# args.exp_code = f'{args.model_name}_{args.exp_date}'
# args.resume_ckpt = "/content/drive/MyDrive/solar_PV_prediction/outputs/segformer-b1_24-10-13/model_19.pth"
# print(args)
# train_csv = os.path.join(args.dataset_csv, "train.csv")
# val_csv = os.path.join(args.dataset_csv, "val.csv")
# test_csv = os.path.join(args.dataset_csv, "test.csv")

# if args.report_to == "wandb":
#   writer = wandb.init(project="Solar-PV-segmentation-finetuning-" + args.exp_code)
# else:
#   writer = None

# training.train_model(train_csv, val_csv, test_csv, writer, args)